### Ingest data on Google Earth Engine (WRI bucket)

* Purpose of script: This notebook will upload the geotiff files from the Google Cloud Storage to the WRI/aqueduct earthengine bucket. 
* Author: Rutger Hofste
* Kernel used: python36
* Date created: 20170802

## Preparation

* Authorize earthengine by running in your terminal: `earthengine authenticate`  
* you need to have access to the WRI-Aquaduct (yep a Google employee made a typo) bucket to ingest the data. Rutger can grant access to write to this folder. 
* Have access to the Google Cloud Storage Bucker

Run in your terminal `gcloud config set project aqueduct30`

In [85]:
import subprocess
import datetime
import os
import time
from datetime import timedelta
import re
import pandas as pd

## Settings

In [2]:
GCS_BASE = "gs://aqueduct30_v01/Y2017M08D02_RH_Upload_to_GoogleCS_V01/"

In [3]:
EE_BASE = "projects/WRI-Aquaduct/PCRGlobWB20V05"

## Functions

In [4]:
def splitKey(key):
    # will yield the root file code and extension of a set of keys
    prefix, extension = key.split(".")
    fileName = prefix.split("/")[-1]
    parameter = fileName[:-12]
    month = fileName[-2:] #can also do this with regular expressions if you like
    year = fileName[-7:-3]
    identifier = fileName[-11:-8]
    outDict = {"fileName":fileName,"extension":extension,"parameter":parameter,"month":month,"year":year,"identifier":identifier}
    return outDict

def splitParameter(parameter):
    #values = parameter.split("_")
    values = re.split("_|-", parameter) #soilmoisture uses a hyphen instead of underscore between the years
    keys = ["geographic_range","temporal_range","indicator","temporal_resolution","units","spatial_resolution","temporal_range_min","temporal_range_max"]
    # ['global', 'historical', 'PDomWN', 'month', 'millionm3', '5min', '1960', '2014']
    outDict = dict(zip(keys, values))
    outDict["parameter"] = parameter
    return outDict


## Script

In [5]:
command = ("earthengine create folder %s") %EE_BASE

In [6]:
print(command)

earthengine create folder projects/WRI-Aquaduct/PCRGlobWB20V05


In [7]:
subprocess.check_output(command,shell=True)

'Asset projects/WRI-Aquaduct/PCRGlobWB20V05 already exists\n'

In [8]:
command = ("/opt/google-cloud-sdk/bin/gsutil ls %s") %(GCS_BASE)

In [9]:
keys = subprocess.check_output(command,shell=True)

In [10]:
keys = keys.decode('UTF-8').splitlines()

Removing first item from the list. The first item contains a folder without file name

In [11]:
keys2 = keys[1:]

In [12]:
df = pd.DataFrame()
i = 0
for key in keys2:
    i = i+1
    outDict = splitKey(key)
    df2 = pd.DataFrame(outDict,index=[i])
    df = df.append(df2)    

In [13]:
df.head()

,extension,fileName,identifier,month,parameter,year
1,tif,global_historical_PDomWN_month_millionm3_5min_...,000,01,global_historical_PDomWN_month_millionm3_5min_...,1960
2,tif,global_historical_PDomWN_month_millionm3_5min_...,001,02,global_historical_PDomWN_month_millionm3_5min_...,1960
3,tif,global_historical_PDomWN_month_millionm3_5min_...,002,03,global_historical_PDomWN_month_millionm3_5min_...,1960
4,tif,global_historical_PDomWN_month_millionm3_5min_...,003,04,global_historical_PDomWN_month_millionm3_5min_...,1960
5,tif,global_historical_PDomWN_month_millionm3_5min_...,004,05,global_historical_PDomWN_month_millionm3_5min_...,1960


In [14]:
df.tail()

,extension,fileName,identifier,month,parameter,year
8545,tif,global_historical_soilmoisture_month_meter_5mi...,679,08,global_historical_soilmoisture_month_meter_5mi...,2014
8546,tif,global_historical_soilmoisture_month_meter_5mi...,680,09,global_historical_soilmoisture_month_meter_5mi...,2014
8547,tif,global_historical_soilmoisture_month_meter_5mi...,681,10,global_historical_soilmoisture_month_meter_5mi...,2014
8548,tif,global_historical_soilmoisture_month_meter_5mi...,682,11,global_historical_soilmoisture_month_meter_5mi...,2014
8549,tif,global_historical_soilmoisture_month_meter_5mi...,683,12,global_historical_soilmoisture_month_meter_5mi...,2014


In [15]:
df.shape

(8549, 6)

In [16]:
parameters = df.parameter.unique()

In [17]:
print(parameters)

[u'global_historical_PDomWN_month_millionm3_5min_1960_2014'
 u'global_historical_PDomWN_year_millionm3_5min_1960_2014'
 u'global_historical_PDomWW_month_millionm3_5min_1960_2014'
 u'global_historical_PDomWW_year_millionm3_5min_1960_2014'
 u'global_historical_PIndWN_month_millionm3_5min_1960_2014'
 u'global_historical_PIndWN_year_millionm3_5min_1960_2014'
 u'global_historical_PIndWW_month_millionm3_5min_1960_2014'
 u'global_historical_PIndWW_year_millionm3_5min_1960_2014'
 u'global_historical_PIrrWN_month_millionm3_5min_1960_2014'
 u'global_historical_PIrrWN_year_millionm3_5min_1960_2014'
 u'global_historical_PIrrWW_month_millionm3_5min_1960_2014'
 u'global_historical_PIrrWW_year_millionm3_5min_1960_2014'
 u'global_historical_PLivWN_month_millionm3_5min_1960_2014'
 u'global_historical_PLivWN_year_millionm3_5min_1960_2014'
 u'global_historical_PLivWW_month_millionm3_5min_1960_2014'
 u'global_historical_PLivWW_year_millionm3_5min_1960_2014'
 u'global_historical_aqbasinwaterstress_month_di

We will store the geotiff images of each NetCDF4 file in imageCollections. The imageCollections will have the same name and content as the original NetCDF4files. 


In [19]:
for parameter in parameters:
    eeLocation = EE_BASE + "/" + parameter
    command = ("earthengine create collection %s") %eeLocation
    # Uncomment the following command if you run this script for the first time
    #subprocess.check_output(command,shell=True)
    #print(command)
    

Now that the folder and collections have been created we can start ingesting the data. It is crucial to store the relevant metadata with the images. 

In [20]:
df_parameter = pd.DataFrame()
i = 0
for parameter in parameters:
    i = i+1
    outDict_parameter = splitParameter(parameter)
    df_parameter2 = pd.DataFrame(outDict_parameter,index=[i])
    df_parameter = df_parameter.append(df_parameter2)   
    

In [21]:
df_parameter.head()

,geographic_range,indicator,parameter,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units
1,global,PDomWN,global_historical_PDomWN_month_millionm3_5min_...,5min,historical,2014,1960,month,millionm3
2,global,PDomWN,global_historical_PDomWN_year_millionm3_5min_1...,5min,historical,2014,1960,year,millionm3
3,global,PDomWW,global_historical_PDomWW_month_millionm3_5min_...,5min,historical,2014,1960,month,millionm3
4,global,PDomWW,global_historical_PDomWW_year_millionm3_5min_1...,5min,historical,2014,1960,year,millionm3
5,global,PIndWN,global_historical_PIndWN_month_millionm3_5min_...,5min,historical,2014,1960,month,millionm3


In [22]:
df_parameter.tail()

,geographic_range,indicator,parameter,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units
19,global,griddedwaterstress,global_historical_griddedwaterstress_month_dim...,5min,historical,2014,1960,month,dimensionless
20,global,griddedwaterstress,global_historical_griddedwaterstress_year_dime...,5min,historical,2014,1960,year,dimensionless
21,global,riverdischarge,global_historical_riverdischarge_month_m3secon...,5min,historical,2014,1960,month,m3second
22,global,riverdischarge,global_historical_riverdischarge_year_m3second...,5min,historical,2014,1960,year,m3second
23,global,soilmoisture,global_historical_soilmoisture_month_meter_5mi...,5min,historical,2014,1958,month,meter


In [23]:
df_parameter.shape

(23, 9)

In [24]:
df_complete = df.merge(df_parameter,how='left',left_on='parameter',right_on='parameter')

Adding NoData value, ingested_by and exportdescription

In [72]:
df_complete["nodata"] = -9999
df_complete["ingested_by"] ="RutgerHofste"
df_complete["exportdescription"] = df_complete["indicator"] + "_" + df_complete["temporal_resolution"]+"Y"+df_complete["year"]+"M"+df_complete["month"]

In [73]:
df_complete.head()

,extension,fileName,identifier,month,parameter,year,geographic_range,indicator,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units,nodata,ingested_by,exportdescription
0,tif,global_historical_PDomWN_month_millionm3_5min_...,000,01,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M01
1,tif,global_historical_PDomWN_month_millionm3_5min_...,001,02,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M02
2,tif,global_historical_PDomWN_month_millionm3_5min_...,002,03,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M03
3,tif,global_historical_PDomWN_month_millionm3_5min_...,003,04,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M04
4,tif,global_historical_PDomWN_month_millionm3_5min_...,004,05,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M05


In [74]:
df_complete.tail()

,extension,fileName,identifier,month,parameter,year,geographic_range,indicator,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units,nodata,ingested_by,exportdescription
8544,tif,global_historical_soilmoisture_month_meter_5mi...,679,08,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M08
8545,tif,global_historical_soilmoisture_month_meter_5mi...,680,09,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M09
8546,tif,global_historical_soilmoisture_month_meter_5mi...,681,10,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M10
8547,tif,global_historical_soilmoisture_month_meter_5mi...,682,11,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M11
8548,tif,global_historical_soilmoisture_month_meter_5mi...,683,12,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M12


In [75]:
list(df_complete.columns.values)

['extension',
 'fileName',
 'identifier',
 'month',
 'parameter',
 'year',
 'geographic_range',
 'indicator',
 'spatial_resolution',
 'temporal_range',
 'temporal_range_max',
 'temporal_range_min',
 'temporal_resolution',
 'units',
 'nodata',
 'ingested_by',
 'exportdescription']

In [76]:
row = df_complete.loc[100]

In [95]:
def uploadEE(row):
    target = EE_BASE +"/"+ row.parameter + "/" + row.fileName
    source = GCS_BASE + row.fileName + "." + row.extension
    metadata = "--nodata_value=%s --time_start %s-%s-01 -p extension=%s -p filename=%s -p identifier=%s -p year=%s -p geographic_range=%s -p indicator=%s -p spatial_resolution=%s -p temporal_range=%s -p temporal_range_max=%s -p temporal_range_min=%s -p temporal_resolution=%s -p units=%s -p ingested_by=%s -p exportdescription=%s" %(row.nodata,row.year,row.month,row.extension,row.fileName,row.identifier,row.year,row.geographic_range,row.indicator,row.spatial_resolution,row.temporal_range,row.temporal_range_max,row.temporal_range_min, row.temporal_resolution, row.units, row.ingested_by, row.exportdescription)
    command = "/opt/anaconda3/bin/earthengine upload image --asset_id %s %s %s" % (target, source,metadata)
    try:
        subprocess.check_output(command, shell=True)
    except:
        print("ERROR!!")
        print(command)
    return 1



In [ ]:
start_time = time.time()
for index, row in df_complete.iterrows():
    elapsed_time = time.time() - start_time 
    print(index,"%.2f" %((index/8548.0)*100), "elapsed: ", str(timedelta(seconds=elapsed_time)))
    uploadEE(row)
    
    
    

(0, '0.00', 'elapsed: ', '0:00:00.003431')
(1, '0.01', 'elapsed: ', '0:00:01.290459')
(2, '0.02', 'elapsed: ', '0:00:02.450219')
(3, '0.04', 'elapsed: ', '0:00:03.624940')
(4, '0.05', 'elapsed: ', '0:00:05.089276')
(5, '0.06', 'elapsed: ', '0:00:06.275719')
(6, '0.07', 'elapsed: ', '0:00:07.455653')
(7, '0.08', 'elapsed: ', '0:00:08.643919')
(8, '0.09', 'elapsed: ', '0:00:09.935281')
(9, '0.11', 'elapsed: ', '0:00:11.083145')
(10, '0.12', 'elapsed: ', '0:00:12.310036')
(11, '0.13', 'elapsed: ', '0:00:13.454741')
(12, '0.14', 'elapsed: ', '0:00:14.636362')
(13, '0.15', 'elapsed: ', '0:00:15.867308')
(14, '0.16', 'elapsed: ', '0:00:17.052000')
(15, '0.18', 'elapsed: ', '0:00:18.263081')
(16, '0.19', 'elapsed: ', '0:00:19.442278')
(17, '0.20', 'elapsed: ', '0:00:20.724541')
(18, '0.21', 'elapsed: ', '0:00:21.939463')
(19, '0.22', 'elapsed: ', '0:00:23.145336')
(20, '0.23', 'elapsed: ', '0:00:24.321772')
(21, '0.25', 'elapsed: ', '0:00:25.679418')
(22, '0.26', 'elapsed: ', '0:00:26.902407'